In [37]:
!pip install python-constraint

### Implementación de Itemsets Frecuentes

utilizaremos técnicas de constraint programming para definir el problema y buscar soluciones que cumplan con las restricciones.





In [38]:
from constraint import *

transactions = [
    ['milk', 'bread', 'beer'],
    ['milk', 'bread'],
    ['bread', 'beer'],
    ['milk', 'beer'],
    ['milk', 'bread', 'beer', 'eggs'],
    ['bread', 'eggs'],
    ['milk', 'eggs']
]

min_support = 0.3

problem = Problem()

items = sorted({item for transaction in transactions for item in transaction})

for item in items:
    problem.addVariable(item, [0, 1])

def count_support(itemset, transactions):
    return sum(1 for transaction in transactions if all(item in transaction for item in itemset))

def min_support_constraint(*args):
    itemset = [items[i] for i, v in enumerate(args) if v == 1]
    return count_support(itemset, transactions) / len(transactions) >= min_support

problem.addConstraint(min_support_constraint, items)

solutions = problem.getSolutions()

frequent_itemsets = [tuple(item for item, v in solution.items() if v == 1) for solution in solutions if any(v == 1 for v in solution.values())]

print("Itemsets frecuentes:", frequent_itemsets)

Itemsets frecuentes: [('beer', 'bread'), ('beer', 'milk'), ('beer',), ('bread', 'milk'), ('bread',), ('eggs',), ('milk',)]


### Explicación de la implementación de Itemsets Frecuentes


1. Definimos las transacciones: Una lista de listas, donde cada sublista representa una transacción.
2. Definimos el soporte mínimo: Un valor que representa el porcentaje mínimo de transacciones en las que un itemset debe aparecer para ser considerado frecuente.
3. Creamos un solver de constraint programming: Utilizamos la biblioteca python-constraint.
4. Definimos las variables: Cada ítem puede estar o no en el itemset (valores 0 o 1).
5. Agregamos restricciones: Contamos el soporte de cada itemset y verificamos si cumple con el soporte mínimo.
6. Resolvemos el problema: Obtenemos todos los itemsets que cumplen con las restricciones y los mostramos.

### Implementación de Reglas de Asociación


In [39]:
from itertools import combinations

min_confidence = 0.7

def calculate_confidence(itemset, antecedent, transactions):
    antecedent_support = count_support(antecedent, transactions)
    if antecedent_support == 0:
        return 0
    itemset_support = count_support(itemset, transactions)
    return itemset_support / antecedent_support

association_rules = []
for itemset in frequent_itemsets:
    for i in range(1, len(itemset)):
        for antecedent in combinations(itemset, i):
            consequent = tuple(item for item in itemset if item not in antecedent)
            if consequent:
                confidence = calculate_confidence(itemset, antecedent, transactions)
                if confidence >= min_confidence:
                    association_rules.append((antecedent, consequent, confidence))

print("Reglas de asociación:")
for rule in association_rules:
    print(f"{rule[0]} -> {rule[1]} (confianza: {rule[2]:.2f})")


Reglas de asociación:
('beer',) -> ('bread',) (confianza: 0.75)
('beer',) -> ('milk',) (confianza: 0.75)


### Explicación de la implementación de Reglas de Asociación

1. Definimos la confianza mínima: Un valor que representa el porcentaje mínimo de veces que la presencia del antecedente implica la presencia del consecuente.
2. Calculamos la confianza de las reglas: Para cada itemset frecuente, generamos todas las posibles reglas de asociación y calculamos su confianza.
3. Filtramos las reglas: Solo guardamos las reglas que cumplen con la confianza mínima.
4. Mostramos las reglas de asociación: Mostramos las reglas que cumplen con las restricciones.

### Implementar una Aplicación Sencilla


Creamos una aplicación sencilla que permita al usuario ingresar transacciones, calcular itemsets frecuentes y generar reglas de asociación.

In [41]:
from constraint import Problem
from itertools import combinations

def count_support(itemset, transactions):
    return sum(1 for transaction in transactions if all(item in transaction for item in itemset))

def calculate_confidence(itemset, antecedent, transactions):
    antecedent_support = count_support(antecedent, transactions)
    if antecedent_support == 0:
        return 0
    itemset_support = count_support(itemset, transactions)
    return itemset_support / antecedent_support

def main():
    with open('transacciones.txt', 'r') as file:
        transactions = [line.strip().split(',') for line in file.readlines() if line.strip()]

    min_support = float(input("Ingrese el soporte mínimo (por ejemplo, 0.3 para 30%): "))
    min_confidence = float(input("Ingrese la confianza mínima (por ejemplo, 0.7 para 70%): "))

    problem = Problem()

    items = sorted({item for transaction in transactions for item in transaction})

    for item in items:
        problem.addVariable(item, [0, 1])

    def min_support_constraint(*args):
        itemset = [items[i] for i, v in enumerate(args) if v == 1]
        return count_support(itemset, transactions) / len(transactions) >= min_support

    problem.addConstraint(min_support_constraint, items)

    solutions = problem.getSolutions()

    frequent_itemsets = [tuple(item for item, v in solution.items() if v == 1) for solution in solutions if any(v == 1 for v in solution.values())]

    print("Itemsets frecuentes:", frequent_itemsets)

    association_rules = []
    for itemset in frequent_itemsets:
        for i in range(1, len(itemset)):
            for antecedent in combinations(itemset, i):
                consequent = tuple(item for item in itemset if item not in antecedent)
                if consequent:
                    confidence = calculate_confidence(itemset, antecedent, transactions)
                    if confidence >= min_confidence:
                        association_rules.append((antecedent, consequent, confidence))

    print("Reglas de asociación:")
    for rule in association_rules:
        print(f"{rule[0]} -> {rule[1]} (confianza: {rule[2]:.2f})")

if __name__ == "__main__":
    main()


Ingrese el soporte mínimo (por ejemplo, 0.3 para 30%): 0.3
Ingrese la confianza mínima (por ejemplo, 0.7 para 70%): 0.7
Itemsets frecuentes: [('beer', 'bread'), ('beer', 'milk'), ('beer',), ('bread', 'milk'), ('bread',), ('eggs',), ('milk',)]
Reglas de asociación:
('beer',) -> ('bread',) (confianza: 0.75)
('beer',) -> ('milk',) (confianza: 0.75)


### Explicación del Proceso y Algoritmos Utilizados

1. Ingreso de Transacciones: Se permite al usuario ingresar transacciones de manera interactiva.
2. Cálculo de Itemsets Frecuentes: Usamos técnicas de constraint programming para encontrar todos los conjuntos de ítems que cumplen con el soporte mínimo.
3. Generación de Reglas de Asociación: A partir de los itemsets frecuentes, generamos todas las posibles reglas de asociación y filtramos aquellas que cumplen con la confianza mínima.
4. Interfaz Interactiva: La aplicación permite al usuario especificar el soporte y la confianza mínimos y muestra los resultados de itemsets frecuentes y reglas de asociación.